based on: https://github.com/joanby/machinelearning-az

# BIC - ANN

The ANN will train on bank information and evaluate if a customer is in danger of exiting the bank.

### Importing the libraries

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf

### Importing the dataset

In [2]:
dataset = pd.read_csv('Churn_Modelling_ANN.csv')
X = dataset.iloc[:, 3:-1].values
y = dataset.iloc[:, -1].values

In [3]:
#Print to identify attributes to encode
print(X)

[[619 'France' 'Female' ... 1 1 101348.88]
 [608 'Spain' 'Female' ... 0 1 112542.58]
 [502 'France' 'Female' ... 1 0 113931.57]
 ...
 [709 'France' 'Female' ... 0 1 42085.58]
 [772 'Germany' 'Male' ... 1 0 92888.52]
 [792 'France' 'Female' ... 1 0 38190.78]]


### Encoding categorical data

#### Label Encoding the "Gender" column

In [4]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
X[:, 2] = le.fit_transform(X[:, 2])

In [5]:
# Confirm encoding and identify labels for male and female
print(X)

[[619 'France' 0 ... 1 1 101348.88]
 [608 'Spain' 0 ... 0 1 112542.58]
 [502 'France' 0 ... 1 0 113931.57]
 ...
 [709 'France' 0 ... 0 1 42085.58]
 [772 'Germany' 1 ... 1 0 92888.52]
 [792 'France' 0 ... 1 0 38190.78]]


#### One Hot Encoding the "Geography" column

In [6]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [1])], remainder='passthrough')
X = np.array(ct.fit_transform(X))

In [7]:
#Identify labels for three countries
print(X)

[[1.0 0.0 0.0 ... 1 1 101348.88]
 [0.0 0.0 1.0 ... 0 1 112542.58]
 [1.0 0.0 0.0 ... 1 0 113931.57]
 ...
 [1.0 0.0 0.0 ... 0 1 42085.58]
 [0.0 1.0 0.0 ... 1 0 92888.52]
 [1.0 0.0 0.0 ... 1 0 38190.78]]


### Split into the Training set and Test set

In [8]:
import sklearn.model_selection as skms
X_train, X_test, y_train, y_test = skms.train_test_split(X, y, test_size = 0.2, random_state = 0)

### Feature Scaling

In [9]:
import sklearn.preprocessing as skpp 
sc = skpp.StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

### Initializing the ANN and adding custom Gaussian, Cosine and Null Activation Functions

In [10]:
ann = tf.keras.models.Sequential()
K = tf.keras.backend
utils = tf.keras.utils
def gaussian(x):
    return K.exp(-K.pow(x,2)/2)
def cosine(x) :
    return K.cos(x)
def nullact(x) :
    return K.constant(0)*x

utils.get_custom_objects().update({'gaussian': tf.keras.layers.Activation(gaussian)})
utils.get_custom_objects().update({'cosine': tf.keras.layers.Activation(cosine)})
utils.get_custom_objects().update({'nullact': tf.keras.layers.Activation(nullact)})

### Number of Neurons and Layers Configuration

In [11]:
neurons = int(input("Enter number of neurons: "))

Enter number of neurons: 6


In [12]:
layers = []
n = int(input("Enter number of layers (1-5): "))
if n < 6 and n > 0:
    for i in range (0, n):
        print("Hyperbolic tangent layer must run every time.")
        print("Available activation functions: linear, sigmoid, gaussian, cosine, null")
        layer = input("Enter name of desired activation function: ")
    
        layers.append(layer)    
else:
    print("Wrong number of layers entered")

Enter number of layers (1-5): 2
Hyperbolic tangent layer must run every time.
Available activation functions: linear, sigmoid, gaussian, cosine, null
Enter name of desired activation function: linear
Hyperbolic tangent layer must run every time.
Available activation functions: linear, sigmoid, gaussian, cosine, null
Enter name of desired activation function: sigmoid


### Adding the input layer and the first hidden layer with linear AF

In [13]:
if "linear" in layers:
    ann.add(tf.keras.layers.Dense(units=neurons, activation='relu'))

### Adding the second hidden layer with Sigmoid AF

In [14]:
if "sigmoid" in layers:
    ann.add(tf.keras.layers.Dense(units=neurons, activation='sigmoid'))

### Adding the third hidden layer with Gaussian AF

In [15]:
if "gaussian" in layers:
    ann.add(tf.keras.layers.Dense(units=neurons, activation='gaussian'))

### Adding the fourth hidden layer with Cosine AF

In [16]:
if "cosine" in layers:
    ann.add(tf.keras.layers.Dense(units=neurons, activation='cosine'))

### Adding the fifth hidden layer with Null AF

In [17]:
if "null" in layers:
    ann.add(tf.keras.layers.Dense(units=neurons, activation='nullact'))

### Adding the output layer with Hyperbolic Tangent AF

In [18]:
ann.add(tf.keras.layers.Dense(units=1, activation='tanh'))

## Training the ANN

### Compiling

In [19]:
ann.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

### Training the ANN on training set

In [20]:
ann.fit(X_train, y_train, batch_size = 32, epochs = 100)

Epoch 1/100
250/250 [==============================] - 0s 441us/step - loss: 0.5463 - accuracy: 0.7836
Epoch 2/100
250/250 [==============================] - 0s 397us/step - loss: 0.4971 - accuracy: 0.7960
Epoch 3/100
250/250 [==============================] - 0s 425us/step - loss: 0.4837 - accuracy: 0.7952
Epoch 4/100
250/250 [==============================] - 0s 504us/step - loss: 0.4623 - accuracy: 0.7954
Epoch 5/100
250/250 [==============================] - 0s 417us/step - loss: 0.4514 - accuracy: 0.7970
Epoch 6/100
250/250 [==============================] - 0s 463us/step - loss: 0.4410 - accuracy: 0.8012
Epoch 7/100
250/250 [==============================] - 0s 447us/step - loss: 0.4383 - accuracy: 0.8054
Epoch 8/100
250/250 [==============================] - 0s 453us/step - loss: 0.4322 - accuracy: 0.8075
Epoch 9/100
250/250 [==============================] - 0s 436us/step - loss: 0.4282 - accuracy: 0.8102
Epoch 10/100
250/250 [==============================] - 0s 448us/step - l

## Evaluating the model

### Single observation prediction

In [21]:
##Check if probability is greater than 50% to determine if customer will exit the bank
print(ann.predict(sc.transform([[1, 0, 0, 600, 1, 40, 3, 60000, 2, 1, 1, 50000]])) > 0.5)

[[False]]


### Predicting the Test set results

In [22]:
y_pred = ann.predict(X_test)
y_pred = (y_pred > 0.5)

### Making the Confusion Matrix

In [23]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

[[1515   80]
 [ 194  211]]


0.863